In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, TrainingArguments, Trainer
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn import functional as F
import torch
from datetime import datetime
from scipy.stats import pearsonr, spearmanr

/usr/lib/python3/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cleaning data

add column names

NOTE : do not run these 3 codes , it would change the csv again but in a wrong way   

In [4]:
###do not run
file_path = 'training.1600000.processed.noemoticon.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1' ,header=None)

column_names = ['label', 'id', 'date', 'query', 'user', 'tweet']  

df.columns = column_names

df.to_csv('training.1600000.processed.noemoticon.csv', index=False)


/tmp/ipykernel_7466/2388293107.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='ISO-8859-1' ,header=None)


terminate duplicated data

In [5]:
###do not run
df.drop_duplicates(subset='id', keep='first', inplace=True)
df.to_csv('training.1600000.processed.noemoticon.csv', index=False)

turn it to lowercase

In [9]:
###do not run
df = df.applymap(lambda x: x.lower() if type(x) == str else x)

run this code if you want to insert csv in to a dataframe for checking codes on continue

In [3]:
file_path = 'training.1600000.processed.noemoticon.csv'
df = pd.read_csv(file_path)


PIPELINE:
Import Libraries: First, import the necessary libraries for text preprocessing and machine learning.

Load the Dataset: Load your dataset containing texts and corresponding labels.

Text Preprocessing:

Tokenization: Split the text into individual words or tokens.
Lowercasing: Convert all words to lowercase to ensure uniformity.
Removing Punctuation: Remove any punctuation marks from the text.
Removing Stopwords: Remove common words that do not contribute much to the meaning of the text (e.g., "the", "is", "and").
Stemming or Lemmatization: Reduce words to their root form to normalize the text. Choose either stemming or lemmatization based on your preference.
Feature Extraction: Convert the preprocessed text into numerical feature vectors using techniques like TF-IDF (Term Frequency-Inverse Document Frequency) or Count Vectorization.

Model Training: Train a machine learning model on the preprocessed text data. This can be a classifier like Naive Bayes, Logistic Regression, or any other suitable algorithm depending on your task.

BOW:

In [4]:
vectorizer_bow = CountVectorizer(max_features=4000)

X_bow = vectorizer_bow.fit_transform(df['tweet'])

feature_names_bow = vectorizer_bow.get_feature_names_out()

print("Feature names (words):")
print(feature_names_bow)
print(len(feature_names_bow))

Feature names (words):
['00' '000' '06' ... 'â½ã' 'â¾' 'â¾ã']
4000


navie:

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(df['tweet'], df['label'], test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

vectorizer_bow = CountVectorizer()

X_train_bow = vectorizer_bow.fit_transform(X_train)
X_valid_bow = vectorizer_bow.transform(X_valid)
X_test_bow = vectorizer_bow.transform(X_test)

param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0]}

nb_classifier = MultinomialNB()

grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train_bow, y_train)

print("Best Hyperparameters:", grid_search.best_params_)

best_classifier = grid_search.best_estimator_

predictions_valid_bow = best_classifier.predict(X_valid_bow)
accuracy_valid_bow = accuracy_score(y_valid, predictions_valid_bow)
print(f"Validation Accuracy: {accuracy_valid_bow:.2f}")

predictions_test_bow = best_classifier.predict(X_test_bow)
accuracy_test_bow = accuracy_score(y_test, predictions_test_bow)
print(f"Test Accuracy: {accuracy_test_bow:.2f}")

print("Classification Report (Test Set):")
print(classification_report(y_test, predictions_test_bow))


Best Hyperparameters: {'alpha': 2.0}
Validation Accuracy: 0.78
Test Accuracy: 0.78
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.76      0.83      0.79     79815
           4       0.81      0.74      0.77     80017

    accuracy                           0.78    159832
   macro avg       0.78      0.78      0.78    159832
weighted avg       0.78      0.78      0.78    159832



TF-IDF:

navie classifier:

In [ ]:
vectorizer_tfidf = TfidfVectorizer(max_features=4000)

X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)

X_test_tfidf = vectorizer_tfidf.transform(X_test)

classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

predictions_tfidf = classifier.predict(X_test_tfidf)

accuracy_tfidf = accuracy_score(y_test, predictions_tfidf)
print(f"Accuracy: {accuracy_tfidf:.2f}")

print("Classification Report:")
print(classification_report(y_test, predictions_tfidf))

feature_names_tfidf = vectorizer_tfidf.get_feature_names_out()

print("Feature names (words):")
print(feature_names_tfidf)

In [21]:
X_train, X_temp, y_train, y_temp = train_test_split(df['tweet'], df['label'], test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

vectorizer_tfidf = TfidfVectorizer(max_features=4000)

X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_valid_tfidf = vectorizer_tfidf.transform(X_valid)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0]}

nb_classifier = MultinomialNB()

grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

print("Best Hyperparameters for Classifier:", grid_search.best_params_)

best_classifier = grid_search.best_estimator_

predictions_valid_tfidf = best_classifier.predict(X_valid_tfidf)
accuracy_valid_tfidf = accuracy_score(y_valid, predictions_valid_tfidf)
print(f"Validation Accuracy (TF-IDF): {accuracy_valid_tfidf:.2f}")

predictions_test_tfidf = best_classifier.predict(X_test_tfidf)
accuracy_test_tfidf = accuracy_score(y_test, predictions_test_tfidf)
print(f"Test Accuracy (TF-IDF): {accuracy_test_tfidf:.2f}")

print("Classification Report (Test Set - TF-IDF):")
print(classification_report(y_test, predictions_test_tfidf))

Best Hyperparameters for Classifier: {'alpha': 2.0}
Validation Accuracy (TF-IDF): 0.77
Test Accuracy (TF-IDF): 0.77
Classification Report (Test Set - TF-IDF):
              precision    recall  f1-score   support

           0       0.76      0.78      0.77     79815
           4       0.78      0.76      0.77     80017

    accuracy                           0.77    159832
   macro avg       0.77      0.77      0.77    159832
weighted avg       0.77      0.77      0.77    159832



logistis classifier:

In [25]:
X_train, X_temp, y_train, y_temp = train_test_split(df['tweet'], df['label'], test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

vectorizer_tfidf = TfidfVectorizer(max_features=4000)

X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_valid_tfidf = vectorizer_tfidf.transform(X_valid)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

param_grid = {
    'C': [0.1, 0.5, 1.0, 2.0],
    'penalty': ['l1', 'l2'],
}

logreg_classifier = LogisticRegression(max_iter=1000)

grid_search = GridSearchCV(estimator=logreg_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

print("Best Hyperparameters for Logistic Regression and Vectorizer:", grid_search.best_params_)

best_logreg_classifier = grid_search.best_estimator_

predictions_valid_tfidf = best_logreg_classifier.predict(X_valid_tfidf)
accuracy_valid_tfidf = accuracy_score(y_valid, predictions_valid_tfidf)
print(f"Validation Accuracy (TF-IDF): {accuracy_valid_tfidf:.2f}")

predictions_test_tfidf = best_logreg_classifier.predict(X_test_tfidf)
accuracy_test_tfidf = accuracy_score(y_test, predictions_test_tfidf)
print(f"Test Accuracy (TF-IDF): {accuracy_test_tfidf:.2f}")

print("Classification Report (Test Set - TF-IDF):")
print(classification_report(y_test, predictions_test_tfidf))


/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise 

Best Hyperparameters for Logistic Regression and Vectorizer: {'C': 1.0, 'penalty': 'l2'}
Validation Accuracy (TF-IDF): 0.79
Test Accuracy (TF-IDF): 0.79
Classification Report (Test Set - TF-IDF):
              precision    recall  f1-score   support

           0       0.80      0.78      0.79     79815
           4       0.78      0.80      0.79     80017

    accuracy                           0.79    159832
   macro avg       0.79      0.79      0.79    159832
weighted avg       0.79      0.79      0.79    159832



logistic for bow:

In [26]:
X_train, X_temp, y_train, y_temp = train_test_split(df['tweet'], df['label'], test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

vectorizer_bow = CountVectorizer(max_features=4000)

X_train_bow = vectorizer_bow.fit_transform(X_train)
X_valid_bow = vectorizer_bow.transform(X_valid)
X_test_bow = vectorizer_bow.transform(X_test)

param_grid = {
    'C': [0.1, 0.5, 1.0, 2.0],
    'penalty': ['l1', 'l2'],
}

logreg_classifier = LogisticRegression(max_iter=1000)

grid_search = GridSearchCV(estimator=logreg_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train_bow, y_train)

print("Best Hyperparameters for Logistic Regression and Bag-of-Words:", grid_search.best_params_)

best_logreg_classifier = grid_search.best_estimator_

predictions_valid_bow = best_logreg_classifier.predict(X_valid_bow)
accuracy_valid_bow = accuracy_score(y_valid, predictions_valid_bow)
print(f"Validation Accuracy (Bag-of-Words): {accuracy_valid_bow:.2f}")

predictions_test_bow = best_logreg_classifier.predict(X_test_bow)
accuracy_test_bow = accuracy_score(y_test, predictions_test_bow)
print(f"Test Accuracy (Bag-of-Words): {accuracy_test_bow:.2f}")

print("Classification Report (Test Set - Bag-of-Words):")
print(classification_report(y_test, predictions_test_bow))

/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise 

Best Hyperparameters for Logistic Regression and Bag-of-Words: {'C': 0.5, 'penalty': 'l2'}
Validation Accuracy (Bag-of-Words): 0.79
Test Accuracy (Bag-of-Words): 0.79
Classification Report (Test Set - Bag-of-Words):
              precision    recall  f1-score   support

           0       0.80      0.77      0.78     79815
           4       0.78      0.81      0.79     80017

    accuracy                           0.79    159832
   macro avg       0.79      0.79      0.79    159832
weighted avg       0.79      0.79      0.79    159832



BERT:

this is the code for fine tuning bert but because of time limit , failed to tuning completely to get the model 

you would make me so thankfull if consider some percent of grade of this section for this code :)

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_classes = 2  
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

train_input_ids = tokenizer(train_df['tweet'].tolist(), truncation=True, padding=True, return_tensors="pt").input_ids
train_labels = torch.tensor(train_df['label'].tolist())

val_input_ids = tokenizer(val_df['tweet'].tolist(), truncation=True, padding=True, return_tensors="pt").input_ids
val_labels = torch.tensor(val_df['label'].tolist())

optimizer = AdamW(model.parameters(), lr=2e-4)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(2):
    outputs = model(train_input_ids, labels=train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        val_outputs = model(val_input_ids, labels=val_labels)
        val_loss = val_outputs.loss

    train_preds = torch.argmax(outputs.logits, dim=1)
    train_accuracy = (train_preds == train_labels).float().mean()

    val_preds = torch.argmax(val_outputs.logits, dim=1)
    val_accuracy = (val_preds == val_labels).float().mean()

    print(f'Epoch {epoch + 1}/{2} - '
          f'Training Loss: {loss.item():.4f} - Training Accuracy: {train_accuracy.item():.4f} - '
          f'Validation Loss: {val_loss.item():.4f} - Validation Accuracy: {val_accuracy.item():.4f}')

model.save_pretrained('fine_tuned_bert_model')

this is for hugging face with the same situatition for bert

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_classes = 2  
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

train_encodings = tokenizer(train_df['tweet'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['tweet'].tolist(), truncation=True, padding=True)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_df['label'].tolist())
val_dataset = CustomDataset(val_encodings, val_df['label'].tolist())

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=2,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset             
)

trainer.train()

trainer.evaluate()

model.save_pretrained('fine_tuned_bert_model')


PART 3

now cause of i dont have bert model , labels in my dataset are via BOW with lack of accuracy

these codes are for caculating correlation 

algorithm:

im getting capitalization of each company and gives tweets and their labels that are for those companies , then i would calculate the percent of 4s labels for each company ; for example if tweets for APPLE have labels like this : 4 0 4 4 4 0 4 0 0 4 , then i will add 60% and the apple capitalization to 2 lists and at the end i will calculate the correlation of these 2 lists

In [5]:
file_path_tweet = 'tweets.csv'
df_tweet = pd.read_csv(file_path_tweet)


In [8]:

X_test_new = df_tweet['text']

X_test_bow_new = vectorizer_bow.transform(X_test_new)

predictions_test_bow_new = best_classifier.predict(X_test_bow_new)

df_tweet['bow_label'] = predictions_test_bow_new

df_tweet.to_csv('tweets.csv', index=False)


In [ ]:
df_tweet['BOW_label'] = df_tweet['bow_label']

In [3]:
tweets_df = pd.read_csv('tweets.csv')
companies_df = pd.read_csv('companies.csv')
entities_df = pd.read_csv('entities.csv')


In [41]:
symbol_entities = entities_df[entities_df['entity_type'] == 'symbol']

top_50_strings = entities_df['text'].unique()[:500]

symbol_entities_subset = symbol_entities[symbol_entities['text'].isin(top_50_strings)]

text_id_mapping = symbol_entities_subset.groupby('text')['tweet_id'].agg(list).to_dict()


In [52]:
companies_df['ids'] = companies_df['ticker'].map(text_id_mapping.get)
good_df = companies_df[companies_df['ids'].notna()]

In [53]:
def map_ids_to_bow_labels(ids_list):
    unique_ids = set(ids_list)
    relevant_tweets = tweets_df[tweets_df['id'].isin(unique_ids)]
    id_to_bow_labels = relevant_tweets.groupby('id')['bow_label'].apply(list).to_dict()
    filtered_ids_list = [id for id in ids_list if id in id_to_bow_labels]
    return [id_to_bow_labels.get(id, None) for id in filtered_ids_list]

good_df['bow_labels'] = good_df['ids'].apply(map_ids_to_bow_labels)

good_df_filtered = good_df[good_df['bow_labels'].apply(lambda x: len(x) > 0)]


/tmp/ipykernel_2717/625022337.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_df['bow_labels'] = good_df['ids'].apply(map_ids_to_bow_labels)


In [62]:
good_df_filtered['percentage_of_4'] = [lst.count([4]) / len(lst) * 100 if len(lst) > 0 else 0 for lst in good_df_filtered['bow_labels']]


In [63]:
list1 = good_df_filtered['percentage_of_4']
list2 = good_df_filtered['capitalization']


In [64]:

correlation_pearson = list1.corr(list2, method='pearson')
correlation_spearman = list1.corr(list2 , method='spearman')

print("Pearson correlation coefficient:", correlation_pearson)
print("Spearman correlation coefficient:", correlation_spearman)

Pearson correlation coefficient: 0.10504229152084112
Spearman correlation coefficient: 0.14671425621318443


correlation in next part :

algorithm:

something like the last part but in 1 monthes , and we can count how many times labels of tweets for each company change , then calculate correlation between all tweets count in 1 month and change counts

In [91]:
def map_ids_to_bow_labels(ids_list):
    unique_ids = set(ids_list)
    relevant_tweets = tweets_df[tweets_df['id'].isin(unique_ids)]
    id_to_bow_labels = relevant_tweets.groupby('id')['created_at'].apply(list).to_dict()
    filtered_ids_list = [id for id in ids_list if id in id_to_bow_labels]
    return [id_to_bow_labels.get(id, None) for id in filtered_ids_list]

good_df['created_at'] = good_df['ids'].apply(map_ids_to_bow_labels)

good_df_filtered = good_df[good_df['created_at'].apply(lambda x: len(x) > 0)]
print(good_df_filtered['created_at'])


1        [[Fri May 19 06:26:11 +0000 2017], [Fri May 19...
39       [[Thu May 18 22:03:25 +0000 2017], [Thu May 18...
59       [[Thu May 18 22:50:06 +0000 2017], [Fri May 19...
65       [[Thu May 18 22:00:12 +0000 2017], [Thu May 18...
182      [[Thu May 18 22:08:20 +0000 2017], [Thu May 18...
                               ...                        
29691    [[Fri May 19 00:07:40 +0000 2017], [Fri May 19...
29707    [[Thu May 18 22:00:22 +0000 2017], [Thu May 18...
29917    [[Sat May 20 08:43:36 +0000 2017], [Tue May 23...
29937    [[Thu May 18 22:48:05 +0000 2017], [Thu May 18...
29988    [[Fri May 19 00:01:25 +0000 2017], [Fri May 19...
Name: created_at, Length: 260, dtype: object


/tmp/ipykernel_2717/2465366264.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_df['created_at'] = good_df['ids'].apply(map_ids_to_bow_labels)


In [98]:
good_df_filtered['created_at'] = good_df_filtered['created_at'].apply(lambda x: [pd.to_datetime(date[0], format='%a %b %d %H:%M:%S %z %Y') for date in x])

print(good_df_filtered['created_at'])

1        [2017-05-19 06:26:11+00:00, 2017-05-19 11:24:4...
39       [2017-05-18 22:03:25+00:00, 2017-05-18 22:49:2...
59       [2017-05-18 22:50:06+00:00, 2017-05-19 02:15:0...
65       [2017-05-18 22:00:12+00:00, 2017-05-18 22:00:2...
182      [2017-05-18 22:08:20+00:00, 2017-05-18 22:13:1...
                               ...                        
29691    [2017-05-19 00:07:40+00:00, 2017-05-19 00:11:2...
29707    [2017-05-18 22:00:22+00:00, 2017-05-18 22:01:1...
29917    [2017-05-20 08:43:36+00:00, 2017-05-23 00:05:1...
29937    [2017-05-18 22:48:05+00:00, 2017-05-18 23:57:3...
29988    [2017-05-19 00:01:25+00:00, 2017-05-19 04:52:0...
Name: created_at, Length: 260, dtype: object


In [108]:
def count_changes(values):
    changes = 0
    prev_value = None
    for value in values:
        for date in value:
            if prev_value is not None:
                if (prev_value == [0] and date == [4]) or (prev_value == [4] and date == [0]):
                    changes += 1
            prev_value = date
    return changes


[0]


In [134]:
flat_dates = [date for sublist in good_df_filtered['created_at'] for date in sublist]

def count_in_month(date_list):
    month_counts = {}
    for date in date_list:
        month = date.month
        if month in month_counts:
            month_counts[month] += 1
        else:
            month_counts[month] = 1
    return [month_counts.get(m, 0) for m in range(5, 10)]  

month_counts = count_in_month(flat_dates)
print(month_counts)

[396674, 1661279, 953900, 847354, 274922]


In [125]:
combined_list = [item for sublist in good_df_filtered['bow_labels'] for item in sublist]
sublist_size = len(combined_list) // 5

sublists = [combined_list[i:i+sublist_size] for i in range(0, len(combined_list), sublist_size)]

remainder = len(combined_list) % 5
for i in range(remainder):
    sublists[i].append(combined_list[-(i+1)])


In [130]:
def count_changes(sublist):
    changes = 0
    for i in range(1, len(sublist)):
        if (sublist[i-1] == [0] and sublist[i] == [4]) or (sublist[i-1] == [4] and sublist[i] == [0]):
            changes += 1
    return changes

counts = [count_changes(sublists[i]) for i in range(5)]

In [137]:
correlation_pearson, _ = pearsonr(month_counts, counts)
correlation_spearman, _ = spearmanr(month_counts, counts)

print("Pearson correlation coefficient:", correlation_pearson)
print("Spearman correlation coefficient:", correlation_spearman)

Pearson correlation coefficient: 0.4586346818643381
Spearman correlation coefficient: 0.6


Both Bag-of-Words (BOW) and TF-IDF (Term Frequency-Inverse Document Frequency) are classic techniques used in text classification. They have their strengths and weaknesses, particularly in terms of where they might make mistakes:

Bag-of-Words (BOW):

Mistakes in Semantic Understanding: BOW treats each word as independent, ignoring word order and context. This can lead to mistakes in capturing the semantic meaning of the text. For example, it might struggle with negations or sarcasm.
Limited Vocabulary: BOW representations often ignore the importance of rare words. If a rare but critical word appears infrequently in the training data, BOW may not assign it enough importance.
Sparse Representation: BOW can result in high-dimensional, sparse feature vectors, especially when dealing with large vocabularies. This can lead to overfitting and make it difficult for classifiers to generalize well, especially with limited training data.

TF-IDF (Term Frequency-Inverse Document Frequency):

Overemphasis on Frequent Terms: While TF-IDF addresses some limitations of BOW by weighing terms by their importance, it still heavily relies on term frequency. If certain terms are overly frequent but not necessarily informative, TF-IDF might assign them high weights, potentially leading to misclassification.
Neglecting Semantic Similarity: TF-IDF, like BOW, doesn't consider semantic relationships between words. Thus, it might struggle with synonyms or terms related to the training data but not explicitly present.
Sensitive to Noise: TF-IDF can be sensitive to noise in the data. If there are mislabeled documents or outliers in the training set, TF-IDF might assign incorrect importance to certain terms, affecting classification accuracy.

Both BOW and TF-IDF can perform well in certain scenarios, particularly when dealing with simple text classification tasks or when computational efficiency is crucial. However, for tasks requiring a deeper understanding of language and context, more advanced techniques like word embeddings or contextual embeddings (e.g., Word2Vec, GloVe, BERT) are often preferred as they capture more nuanced semantic relationships between words and phrases.

Transfer Learning:
Transfer learning involves leveraging knowledge gained from training a model on one task and applying it to a different but related task. Instead of training a model from scratch, which requires a large amount of labeled data and computational resources, transfer learning allows us to use pre-trained models that have been trained on large datasets for general tasks like image classification, text generation, or language understanding.

Advantages of Transfer Learning:
Faster Training: Pre-trained models have already learned general patterns from massive datasets, reducing the amount of training time required for the target task.
Better Performance with Limited Data: Transfer learning enables the utilization of knowledge from large datasets, even when the target task has limited labeled data.
Generalization: Transfer learning helps models generalize better to new tasks or domains, as they have learned robust features from diverse data.

Fine-Tuning:
Fine-tuning is a specific application of transfer learning where a pre-trained model is further trained on a new dataset or task with specific modifications to adapt it to the new task. Instead of retraining the entire model, fine-tuning involves adjusting the parameters of the pre-trained model to better fit the new task.

Advantages of Fine-Tuning:
Task-Specific Adaptation: Fine-tuning allows the model to adapt to the nuances of the target task by adjusting its parameters while retaining the knowledge learned during pre-training.
Improved Performance: Fine-tuning often leads to improved performance on the target task compared to using the pre-trained model without further adaptation.
Cost-Effective: Fine-tuning requires less computational resources compared to training a model from scratch since the pre-trained model provides a good starting point.

Transfer learning and fine-tuning can be applied in various projects across different domains:

Computer Vision: Pre-trained convolutional neural networks (CNNs) like VGG, ResNet, or EfficientNet can be fine-tuned for specific image classification or object detection tasks.

Natural Language Processing (NLP): Pre-trained language models like BERT, GPT, or RoBERTa can be fine-tuned for tasks such as sentiment analysis, text classification, or named entity recognition.

Healthcare: Transfer learning can be applied to medical imaging tasks, where pre-trained models on natural images (e.g., ImageNet) are fine-tuned on medical image datasets for tasks like disease diagnosis or tumor detection.

Recommendation Systems: Transfer learning can be used in recommendation systems by leveraging pre-trained embeddings or models trained on large-scale datasets to improve the performance of personalized recommendation algorithms.

Autonomous Driving: Transfer learning can be applied to perception tasks in autonomous driving, where pre-trained models on general driving scenes are fine-tuned on specific environments or driving conditions to enhance performance and safety.

